**Name: Hang Chen**

**Student Number: N18107496**

**Class: Artificial Intelligence**

Import some libs and prepare for the kernal.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Read Original Data

In [ ]:
X_train = pd.read_csv('/kaggle/input/career-con-2019/X_train.csv')
#X_test = pd.read_csv('/kaggle/input/career-con-2019/X_test.csv')
y_train = pd.read_csv('/kaggle/input/career-con-2019/y_train.csv')
#submission = pd.read_csv('/kaggle/input/career-con-2019/sample_submission.csv')
data_train = pd.read_csv('/kaggle/input/252-features/data_252_features.csv')
print('Read Finished')

Feature Engineering:
* Leakage Analysis:
* According to my explore in orientation data and the [Discussion](https://www.kaggle.com/c/career-con-2019/discussion/87239#latest-508136), I found that there is a very strong correlation between orientation data and surface. To be specific, Series m and Series n have similiar orientation data (vary a little), it turns out that the Series m and Series n are in high probability to have same surface.
* So this phenomenon will cause that if we split our train data into train and val datasets and do not deal with orientation data, we will find quiet high CV score because we only hide the label of val data but did not hide the orientation data.
* In me below code I check my guess with two different dataset. One is the whole dataset  including orientation data (Dataset 1) and the other is dataset without any orientation data (Dataset 2). I found that if we use Dataset 1 we can easily gei CV score (acc) higher that 0.90, but if we use Dataset 2 our CV score is around 0.60. However, there is an explian that "maybe it means that the orientation data is an very important feature and if you drop them you can definitly get lower score!". To check it I must submit my result to Kaggle platform to get my public leaderboard score. Unluckily, there must be something wrong with Kaggle evaluate system.... So I refer to these discussions and kernals:

* https://www.kaggle.com/c/career-con-2019/discussion/84243

* https://www.kaggle.com/c/career-con-2019/discussion/84434

* https://www.kaggle.com/friedchips/the-missing-link

* https://www.kaggle.com/anjum48/leakage-within-the-train-dataset

* So it is quiet easy to get high score in CV but there will be a huge drop between CV and public leaderboard. **It means that if we evaluate our model performance with Dataset 1 it is useless. Dataset 2 will have a true reflection of your model performance.**

* Here I generated several features including basic features and time series features.

**Basic Features:**
1. Mean
2. Mode
3. Median
4. Max
5. Min
6. Range
7. Sum Acceleration of Linear
8. Sum Acceleration of Angular

**Time Series Features**
1. Absolute Energy: Describes the squared transient from the origin of time series data
2. Absolute Sum of Changes: Describe absolute fluctuations between adjacent observations of time series data
3. Agg Autocorrelation: Aggregate characteristics of differences between time series data. Describe different characteristics of time series data according to different aggregation functions
4. Approximate Entropy: Ratio between adjacent entropy values
5. Augmented Dickey Fuller: ADF test statistics
6. Binned Entropy: Isometric grouping of time series data for entropy
7. Cid Ce: Used to evaluate the complexity of time series. The more complex the series, the more valleys
8. FFT: Fast Fourier Transform coefficient
9. Kurtosis: Calculate kurtosis based on modified Fisher-Pearson moment statistics

In this project, each measurement contains 128 sample point which is time serries data.So I apply some time series algorithm to generate a high level time period features such as fast fourier transform. These time series features mainly focus on:
* Fluctuation during the whole time
* Entrophy of the serie
* Complexity of the serie
* Robust ability of value in the period
* Transformation extract more information
* Statis
* Correlation of each sample

In [ ]:
import tsfresh as tsf

def feature_engineer(df):
    measure_length = 128 
    res = pd.DataFrame()
    for i in tqdm(range(int(len(df)/measure_length))):
        ans_df = pd.Series()
        sub_df = df.iloc[i*128:(i+1)*128].copy()
        sub_df['sum_acc_linear'] = sum([sub_df['linear_acceleration_X']**2, sub_df['linear_acceleration_Y']**2, sub_df['linear_acceleration_Z']**2])**0.5
        sub_df['sum_acc_angular'] = sum([sub_df['angular_velocity_X']**2, sub_df['angular_velocity_Y']**2, sub_df['angular_velocity_Z']**2])**0.5
        for each in sub_df.columns:
            if each in ['row_id','series_id','measurement_number']:
                continue
            ans_df[each+'_mean'] = sub_df[each].mean()
            ans_df[each+'_mode'] = sub_df[each].mode()[0]
            ans_df[each+'_median'] = sub_df[each].median()
            ans_df[each+'_max'] = sub_df[each].max()
            ans_df[each+'_min'] = sub_df[each].min()
            ans_df[each+'_range'] = ans_df[each+'_max'] - ans_df[each+'_min']
            ans_df[each+'_std'] = sub_df[each].std()
            ans_df[each+'_mean'] = sub_df[each].mean()
            ans_df[each+'count_above_mean'] = tsf.feature_extraction.feature_calculators.count_above_mean(sub_df[each])
            ans_df[each+'count_below_mean'] = tsf.feature_extraction.feature_calculators.count_below_mean(sub_df[each])
            
            ans_df[each+'_abs_energy'] = tsf.feature_extraction.feature_calculators.abs_energy(sub_df[each])
            ans_df[each+'_abs_sum_changes'] = tsf.feature_extraction.feature_calculators.absolute_sum_of_changes(sub_df[each])
            ans_df[each+'_agg_autocorrelation'] = tsf.feature_extraction.feature_calculators.agg_autocorrelation(sub_df[each], [{'f_agg': 'mean', 'maxlag':2}])[0][1]
            ans_df[each+'_approximate_entropy'] = tsf.feature_extraction.feature_calculators.approximate_entropy(sub_df[each], 16, 0.05)
            ans_df[each+'_augmented_dickey_fuller'] = tsf.feature_extraction.feature_calculators.augmented_dickey_fuller(sub_df[each], [{'attr': 'pvalue'}])[0][1]
            ans_df[each+'_binned_entropy'] = tsf.feature_extraction.feature_calculators.binned_entropy(sub_df[each], 16)
            ans_df[each+'_cid_ce'] = tsf.feature_extraction.feature_calculators.cid_ce(sub_df[each], True)
            ans_df[each+'_fft_real'] = list(tsf.feature_extraction.feature_calculators.fft_coefficient(sub_df[each], [{'coeff': 2, 'attr': 'real'}]))[0][1]
            ans_df[each+'_fft_imag'] = list(tsf.feature_extraction.feature_calculators.fft_coefficient(sub_df[each], [{'coeff': 2, 'attr': 'imag'}]))[0][1]
            ans_df[each+'_fft_abs'] = list(tsf.feature_extraction.feature_calculators.fft_coefficient(sub_df[each], [{'coeff': 2, 'attr': 'abs'}]))[0][1]
            ans_df[each+'_fft_angle'] = list(tsf.feature_extraction.feature_calculators.fft_coefficient(sub_df[each], [{'coeff': 2, 'attr': 'angle'}]))[0][1]
            ans_df[each+'_kurtosis'] = tsf.feature_extraction.feature_calculators.kurtosis(sub_df[each])
            
        res[i] = ans_df.T
    return res.T

try:
    print('Using feature engineered data.')
    print('data_train shape: {}'.format(data_train.shape))
except:
    print('no data train,start feature engineering')
    data_train = feature_engineer(X_train)
    print('Feature Engineering Finished')



Split Train and Test Dataset
Details:
1. I apply a simple LableEncoder on y_train['surface'] which is categorical label.
2. I apply my feature_engineer function on X_train dataset to generate 168 new features

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer

data_train_with_ori = data_train.copy()
data_train_without_ori = data_train.copy()
for each_feature in data_train.columns:
    if 'orientation' in each_feature:
        data_train_without_ori.drop(each_feature, axis = 1, inplace = True)
print('ori_data_train shape: {}, type: {}\n\nwithoutori_data_train shape:{}, type: {}\n'.format(data_train_with_ori.shape, type(data_train_with_ori), data_train_without_ori.shape, type(data_train_without_ori)))



le = LabelEncoder()
y_train['surface'] = le.fit_transform(y_train['surface'])
data_label = y_train['surface']
data_label = data_label.astype(np.int8)
print('data_label shape: {}, type: {}\n'.format(data_label.shape, type(data_label)))


Save Data after Feature Engineering
I need to save my feature engineered data matrix because it takes too long (around 1.5 hours) to finished operating feature engineering function. In the kernal, I will read the feature engineered data for the next use.

In [ ]:
#save_data = pd.concat([data_train, data_label], axis = 1)
#data_train.to_csv('/kaggle/working/data_252_features.csv', index = False)

Start split the dataset into train and test

Details:

I lock the seed with 2020 and the test_size is 30% of all dataset.

* I apply Normalization which make the data more friendly to the model and I choose L2 norm to avoid overfitting. Attention Here, why I do not apply Normalizer on the whole dataset? It's because we need to seperate train and test data. **If we apply normalization algorithm to the whole dataset, it convey the imformation of test data to train data via the normalization formula below.** Value_normalize_feature1 = Value_origional_feature1 / sum(Value of feature1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_with_ori, X_test_with_ori, y_train_with_ori, y_test_with_ori = train_test_split(data_train_with_ori, data_label, test_size=0.3,random_state=0)
X_train_without_ori, X_test_without_ori, y_train_without_ori, y_test_without_ori = train_test_split(data_train_without_ori, data_label, test_size=0.3,random_state=0)
print('with_ori: {}, {}, {}, {}'.format(X_train_with_ori.shape, X_test_with_ori.shape, y_train_with_ori.shape, y_test_with_ori.shape))
print('without_ori: {}, {}, {}, {}'.format(X_train_without_ori.shape, X_test_without_ori.shape, y_train_without_ori.shape, y_test_without_ori.shape))

def normalize(data_train):
    normalizer = Normalizer()
    data_train = normalizer.fit_transform(data_train)
    return data_train

X_train_with_ori = normalize(X_train_with_ori)
X_test_with_ori = normalize(X_test_with_ori)
X_train_without_ori = normalize(X_train_without_ori)
X_test_without_ori = normalize(X_test_without_ori)
#print('ori type: {}, shape: {}\nno_ori type: {}, shape: {}'.format(type(data_train_with_ori), data_train_with_ori.shape, type(data_train_without_ori), data_train_without_ori.shape))

Training

Details:

Here I choose RandomForestClassifier (RFC) and LightGBM (LGB) as my preference model.

Brief Intro of RFC:

It's a classical bagging algorithm. It's composed by many Decision Trees. Every estimator will select m features with replacement in total N features to compose a Decision Tree which have a prediction. And the RFC will select the result with the highest vote of all estimators.
Why RFC:
* It is very robust in Kaggle Classification Competition which can easily be regarded as start baseline.
* It's fast for experiment iteration and low in memory occupation.
* When in RFC algorithm, we need to split features, it take information gain into consideration (e.g. information entrophy), that could reflect feature importance.
* It's not easy to overfit.
* For unbalanced data sets, it can balance errors.



Bried Intro of LGB:

In NIPS 2017, Microsoft represented a light Grandient Boosting Decision Tree framework called LightGBM. All these boosting algorithm will focus on max information gain to find split point of features. LGB only needs a small number of samples to calculate the information gain, so it will be much faster. In order to minimize the information loss, LGB selects samples with large gradients to calculate the information gain. At the same time, some small gradient samples are kept randomly, and the information gain brought by the small gradient samples is amplified.
According to some application of LGB in Kaggle, this algorithm express impressive speed and accuracy in many task.



After several experiments (including other classifiers such as SVM, FullyConnected Neural Network etc.), I foud that the ensemble RandomForestClassifier(RFC) and LightGBMClassifier(LGB) with 7 folds Cross Validation (CV) will have the best performance. The ensemble formula is:

* final_pred_prob = RFC_weights * RFC_pred_prob + LGB_wights * LGB_pred_prob
* RFC_weights = sum_of_each_fold_CV_RFC_acc / k_folds
* LGB_weights = sum_of_each_fold_CV_LGB_acc / k_folds
* RFC_pred_prob = sum_of_each_fold_CV_RFC_pred_prob / k_folds
* LGB_pred_prob = sum_of_each_fold_CV__pred_prob / k_folds

Simply speaking: final prediction is voted by two Classifier. Each weight is there average acc in CV.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
def k_folds(X, y, X_test, k):
    clf_dic = {'RFC': RandomForestClassifier(),
               'LGB': lgb.LGBMClassifier(objective='multiclass')}
    res = pd.DataFrame()
    ensemble_test = np.zeros((X_test.shape[0], 9))
    X = np.array(X)
    y = np.array(y)
    X_test = np.array(X_test)
    for each in list(clf_dic.keys()):
        print('Training {}'.format(each))
        folds = StratifiedKFold(n_splits = k, shuffle=True, random_state=2019)
        y_test = np.zeros((X_test.shape[0], 9))
        y_oof = np.zeros((X.shape[0]))
        score = 0
        for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
            in_loop_train_data = X[train_idx]
            in_loop_train_label = y[train_idx]
            in_loop_val_data = X[val_idx]
            in_loop_val_label = y[val_idx]
            clf =  clf_dic[each]
            clf.fit(in_loop_train_data, in_loop_train_label)
            
            y_oof[val_idx] = clf.predict(in_loop_val_data)
            y_test += clf.predict_proba(X_test)
            score += clf.score(in_loop_val_data, in_loop_val_label)
            print('clf: {} Fold: {} score: {}'.format(each,i+1,clf.score(in_loop_val_data, in_loop_val_label)))
        print('{} Avg Accuracy {}'.format(each, score/folds.n_splits))
        res[each] = np.argmax(y_test, axis=1)
        ensemble_test += (y_test / folds.n_splits) * (score/folds.n_splits)
        print('\n\n')
    return res, ensemble_test
print('Using ori data')
res_with_ori, ensemble_test_with_ori = k_folds(X_train_with_ori, y_train_with_ori, X_test_with_ori, k = 7)
print('Using no ori data')
res_without_ori, ensemble_test_without_ori = k_folds(X_train_without_ori, y_train_without_ori, X_test_without_ori, k = 7)

Evaluate my result
Actuacally I could not break to higher 0.6 without orientation data. Although we drop the leakage inforamtion, meanwhile we also drop some useful information that may contribute a lot.

In [ ]:
from sklearn.metrics import accuracy_score
final_with_ori = np.argmax(ensemble_test_with_ori, axis=1)
final_without_ori = np.argmax(ensemble_test_without_ori, axis=1)
acc_with_ori = accuracy_score(final_with_ori, y_test_with_ori)
acc_without_ori = accuracy_score(final_without_ori, y_test_without_ori)
print('Use orientation data test acc: {}\nNot Use orientation data test acc: {}'.format(acc_with_ori, acc_without_ori))